In [1]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, DuckDuckGoSearchTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [7]:
# from google.colab import userdata
# import os
# os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [8]:
agent = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [9]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
The London location known as Two Temple Place served as another The Batman filming location. Two Temple Place was 
the first building seen in the movie, in fact, as it was the home of Gotham's Mayor Don Mitchell. Matt Reeves began
The Batman by showing the location through Riddler's eyes. Two Temple Place then became a major location for the 
...

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022-2.php)
Please wait while your request is being verified...

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
Matt Reeves' The Batman was filmed in many places around the globe to create a new version of Gotham and used 
real-life locations to capture the right look. This new take on the Dark Knight introduced audiences to Robert 
Pattinson's Batman and the environment he lives in. Along with bringing characters like Penguin, Catwoman, and 
others from DC Comics to life, the movie truly made Gotham ...

[These Are the Cities Standing in for Gotham in 'The Batman' - 
Distractify](https://www.distractify.com/p/the-batman-filming-locations)
The latest film centered around the Caped Crusader, The Batman, is finally in theaters.One of the key elements of 
any good Batman story is the Gotham City setting. Over the years, various directors have chosen different cities to
stand in for the crime-riddled location, but where did Matt Reeves set his epic Batman tale?

[Where Was The Batman Filmed? 2022 Movie Filming 
Locations](https://thecinemaholic.com/where-was-the-batman-filmed/)
The Batman Filming Locations 'The Batman' was filmed in various parts of England and Scotland, specifically in 
Liverpool, London, Watford, Bedford, Glasgow, and Shotts. Some portions were also filmed in Illinois, particularly 
in Chicago. Principal photography for the movie commenced on January 27, 2020, but was abruptly halted on March 14 
...

[Check Out These The Batman Filming Locations For A Glimpse Of Gotham 
City](https://www.elitedaily.com/lifestyle/where-was-the-batman-filmed-2022-filming-locations)
2022's 'The Batman' premieres in theaters on March 4. Fans of film may be wondering, where was 'The Batman' filmed?
Here are 'The Batman' 2022 filming locations you can visit in real life.

[The Batman Filming Locations: Where The Batcave Is Located? - 
OtakuKart](https://otakukart.com/the-batman-filming-locations-where-the-batcave-is-located/)
Marshall Suloway Bridge, situated in Chicago, became a part of the Filming locations of The Batman when we saw 
Bruce riding his motorcycle on the Bridge. Even Marshall Suloway Bridge is one of the historical locations in 
Chicago because it was built back in 1928 by a steel company investing $2,500,000. So, if

[Step 0: Duration 21.18 seconds| Input tokens: 2,377 | Output tokens: 160]

...

[Step 3: Duration 27.12 seconds| Input tokens: 16,766 | Output tokens: 1,740]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(combined_df)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                            Location  Travel Time (hours)
0          Two Temple Place, London                 9.17
1                     Liverpool, UK                 8.81
2                  Chicago, IL, USA                 2.68
3                       Watford, UK                 9.14
4                       Bedford, UK                 9.11
5                       Glasgow, UK                 8.60
6                        Shotts, UK                 8.64
7  Marshall Suloway Bridge, Chicago                 2.68
8                     Edinburgh, UK                 8.69
9               McLaren, Woking, UK                 9.14

[Step 4: Duration 1.83 seconds| Input tokens: 24,423 | Output tokens: 1,791]

In [10]:
result

,Location,Travel Time (hours)
0,"Two Temple Place, London",9.17
1,"Liverpool, UK",8.81
2,"Chicago, IL, USA",2.68
3,"Watford, UK",9.14
4,"Bedford, UK",9.11
5,"Glasgow, UK",8.60
6,"Shotts, UK",8.64
7,"Marshall Suloway Bridge, Chicago",2.68
8,"Edinburgh, UK",8.69
9,"McLaren, Woking, UK",9.14


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [11]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location found, visit the source URL to confirm the details and coordinates.
3. Perform a web search to find a comprehensive list of supercar factories worldwide.
4. For each supercar factory found, visit the source URL to confirm the details and coordinates.
5. Calculate the cargo plane transfer time from Gotham to each confirmed Batman filming location.
6. Calculate the cargo plane transfer time from Gotham to each confirmed supercar factory.
7. Filter the Batman filming locations based on the calculated transfer time.
8. Filter the supercar factories based on the calculated transfer time.
9. Organize the filtered Batman filming locations and supercar factories into a pandas dataframe.
10. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.         
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code parsing failed on line 1 due to: SyntaxError
Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.
            ^
Error: invalid syntax (<unknown>, line 1)

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions
(Request ID: Root=1-67c2dd74-6807d0b008e2f78378472430;c1c9c249-1515-431c-91d4-8f813d2b1722)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
allowance.

[Step 3: Duration 0.13 seconds| Input tokens: 15,192 | Output tokens: 520]

[Step 4: Duration 0.47 seconds| Input tokens: 19,713 | Output tokens: 709]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://huggingface.co/api/inference-proxy/together/v1/chat/completions (Request ID: Root=1-67c2dd74-2f4138ba4389d9953b48370e;1fb75fc5-0190-4557-8ec7-7c048442934e)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.

In [ ]:
detailed_report

,Location,Travel Time (hours)
0,"Bridge of Sighs, Glasgow Necropolis, Glasgow, ...",8.6
1,"Wishart Street, Glasgow, Scotland, UK",8.6


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                <think>
Okay, let's break down the problem step by step. The user wants a map showing Batman filming locations and supercar
factories with their cargo plane transfer times to Gotham. The total points should be at least six, with supercar 
factories having the same transfer times as some Batman locations.

First, I need to gather the Batman filming locations. Since I don't have that data, I'll need to use the web_agent 
to search for reliable sources. Once I have the list of locations with their coordinates, I can calculate the 
travel time from each to Gotham using the calculate_cargo_travel_time tool. 

Next, I need to find supercar factories. Again, the web_agent can help find their locations. For each factory, I'll
calculate the travel time to Gotham. Then, I have to match these times with the Batman locations' times. If a 
factory's time matches any of the Batman times, it gets included. If there aren't enough matches, I might need to 
adjust by finding more factories or relaxing the time matching criteria, but the user specified "same time," so 
I'll stick to exact matches as much as possible.

Once I have at least six points (combination of Batman locations and matching factories), I'll compile them into a 
dataset. The example code uses a DataFrame with lat, lon, name, and color based on travel time. I'll structure my 
data similarly. Finally, I'll plot the points on a map using plotly, color-coding by travel time, and save the 
image.
</think>

1. Task the web agent to compile a list of all Batman movie filming locations worldwide, including their geographic
coordinates (latitude/longitude).  
2. For each Batman filming location, use the `calculate_cargo_travel_time` tool to determine the cargo plane 
transfer time to Gotham (40.7128° N, 74.0060° W).  
3. Task the web agent to identify factories of supercar manufacturers globally, including their geographic 
coordinates.  
4. For each supercar factory, calculate its cargo plane transfer time to Gotham using the same tool.  
5. Filter the supercar factories to retain only those with transfer times matching the Batman locations’ calculated
times. Combine these with Batman locations to ensure at least 6 total points.  
6. Create a DataFrame with columns for latitude, longitude, location/factory name, and transfer time.  
7. Plot the combined data on a global map using `plotly.express`, coloring points by transfer time. Save the map to
`saved_map.png`.  
8. Return the saved map as the final answer.  


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
 [STEP 1]:                                                                                                         
Thought: I need to get the list of Batman filming locations with their coordinates. Since I don't have this data,  
I'll task the web_agent to search for it.                                                                          
Code:                                                                                                              
```py                                                                                                              
batman_locations_task = "Find all Batman movie filming locations worldwide. For each location, provide the name and
geographic coordinates (latitude, longitude). Use reliable sources like IMDb or Movie-Locations.com."              
web_agent_response = web_agent(task=batman_locations_task)                                                         
print(web_agent_response)                                                                                          
```<end_code>                                                                                                      
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_task = "Find all Batman movie filming locations worldwide. For each location, provide the name  
  and geographic coordinates (latitude, longitude). Use reliable sources like IMDb or Movie-Locations.com."        
  web_agent_response = web_agent(task=batman_locations_task)                                                       
  print(web_agent_response)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Reconstruct batman_times with original structure (from Step 2)                                                 
  batman_data = [                                                                                                  
      ("Stearns Wharf, Santa Barbara", 34.4188, -119.6951, 6.93),                                                  
      ("Cliffs, Marineland", 33.9, -118.4, 6.80),                                                                  
      ("Bronson Caves, Griffith Park", 34.1347, -118.3215, 6.78),                                                  
      ("Alcatraz Island", 37.8267, -122.4233, 7.05),                                                               
      ("Knebworth House, UK", 51.8055, -0.4035, 9.13),                                                             
      ("Skaftafell, Iceland", 64.2128, -19.0649, 7.37),                                                            
      ("Chicago Board of Trade", 41.8879, -87.6235, 2.68),                                                         
      ("St Pancras Hotel, London", 51.5343, -0.1341, 9.17),                                                        
      ("Battersea Power Station", 51.4719, -0.1167, 9.17),                                                         
      ("Necropolis Cemetery", 55.8642, -4.2518, 8.60)                                                              
  ]                                                                                                                
                                                                                                                   
  supercar_data = [                                                                                                
      ("Ferrari Maranello", 44.58, 10.89, 10.72),                                                                  
      ("Lamborghini Sant'Agata", 44.54, 11.18, 10.75),                                                             
      ("Bugatti Molsheim", 48.69, 7.71, 10.10)                                                                     
  ]                                                                                                                
                                                                                                                   
  # Combine into DataFrame                                                                                         
  import pandas as pd                                                                                              
  df = pd.DataFrame(                                                                                               
      [                                                                                                            
          {                                                                                                        
              'name': name,                                                                                        
              'lat': lat,                                                                                          
              'lon': lon,                                                                                          
              'time': time,                                                                                        
              'type': 'Batman Location'                                                                            
          } for (name, lat, lon, time) in batman_data                                                              
      ][38;2;248;248;

Out - Final answer: saved_map.png

Feedback:  **Reasons Why Yes:**

1. **Plotting Method**: The plot was made using `px.scatter_mapbox`, which is the correct method as per the task requirements.
2. **Location Representation**: The map shows both Batman filming locations and supercar factories, fulfilling the request for a spatial representation.
3. **Color Coding**: Locations are color-coded based on travel time, providing a clear visual differentiation.
4. **Number of Points**: The plot displays at least six points, meeting the minimum requirement set by the task.

**Reasons Why No:**

1. **Matching Times**: The task required supercar factories to match Batman locations' transfer times. This plot doesn't specifically confirm any matching times.
2. **Transfer Time Calculation**: There was a logical error in handling the transfer times that wasn't corrected in plotting. 
3. **Variable Errors**: The plot creation initially faced issues due to handling errors in data structures, affecting data integrity.

**Final Decision:

[Step 6: Duration 37.00 seconds| Input tokens: 34,990 | Output tokens: 2,560]

'saved_map.png'

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)